<a href="https://colab.research.google.com/github/bradsprigg/prompt-explorer/blob/main/Prompt_Explorer_v0_5_Brad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Explorer v0.5 by Sathias
Based on Stable Diffusion notebook by [@pharmapsychotic](https://twitter.com/pharmapsychotic)

Original notebook available at https://colab.research.google.com/github/pharmapsychotic/ai-notebooks/blob/main/pharmapsychotic_Stable_Diffusion.ipynb#scrollTo=wzmVAdZ1-5tE

Version History<br>
0.1 Quick and Dirty Version<br>
0.2 Rewrite based on pharmapsychotic notebook<br>
0.3 More configurability options in prompt usage<br>
0.4 Added Prompt Randomizer code by @ai_curio as an extra option<br>
0.5 Updated to include prompt weights and negative prompts<br>

TO-DO
*   Cleaner prompt selection code
*   Logging of Prompts
*   Prompt Weighting support
*   SWINIR upsizing
*   Kaggle dataset integration
*   Configurable delimiters
*   CLIP Interrogator integration
*   Recursive img2img with video support
*   Twitter integration
*   Wonderwords integration https://github.com/mrmaxguns/wonderwordsmodule




In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b9d883ee-7aa3-71b2-0d84-1ebd63337e9a)


In [2]:
#@title Mount Google Drive and Prepare Folders
from google.colab import drive
drive.mount('/content/gdrive')
outputs_path = "/content/gdrive/MyDrive/AI/StableDiffusion"
!mkdir -p $outputs_path
print(f"Outputs will be saved to {outputs_path}")

Mounted at /content/gdrive
Outputs will be saved to /content/gdrive/MyDrive/AI/StableDiffusion


In [3]:
#@title Installation
!pip install pytorch-lightning torch-fidelity
!pip install numpy omegaconf einops kornia pytorch-lightning
!pip install albumentations transformers
!pip install ftfy jsonmerge resize-right torchdiffeq tqdm

!git clone https://github.com/first20hours/google-10000-english
!git clone https://github.com/CompVis/stable-diffusion
%cd stable-diffusion/
!git clone https://github.com/CompVis/taming-transformers
!git clone https://github.com/openai/CLIP.git
!git clone https://github.com/crowsonkb/k-diffusion.git

import sys
sys.path.append(".")
sys.path.append("./CLIP")
sys.path.append('./taming-transformers')
sys.path.append('./k-diffusion')

!echo '' > ./k-diffusion/k_diffusion/__init__.py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 708 kB 4.1 MB/s 
     |████████████████████████████████| 529 kB 73.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 551 kB 23.6 MB/s 
     |████████████████████████████████| 117 kB 87.4 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=fa916d38e52492f55ffc93fa835f39b57012bd21f49d6da5629ed5b490c0e89e
  Stored in directory: /root/.cache/pip/wheels/8b/8d/53/2af8772d9aec614e3fc65e53d4a993ad73c61daa8bbd85a873
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.3 MB/s 
     |████████████████████████████████| 7.6 MB 54.3 MB/s 
     |████████████████████████████████| 163 kB 77.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.3 MB/s 
  Created wheel for jsonmerge: filename=jsonmerge-1.8.0-py3-none-any.whl size=18013 sha256=98bcb7a45788de9c339d3948dccb2af749990206da1b49dc36856fca8dd8bc9c
  Stored in directory: /root/.cache/pip/wheels/2f/c8/79/83ddc70e0b20f2df3bbac658c2c5d665b76cedd02e67bd61dc
Successfully built jsonmerge
Cloning into 'google-10000-english'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 93 (delta 1), reused 3 (delta 0), pack-reused 87
Unpacking objects: 100% (93/93), done.
Cloning in

In [ ]:
#@title Load Model

#@markdown You need to get the model weights yourself and put on Google Drive or this Colab instance
checkpoint_model_file = "/content/gdrive/MyDrive/stable-diffusion/sd-v1-4.ckpt" #@param {type:"string"}

import argparse, gc, json, os, random, sys, time, glob, requests
import torch
import torch.nn as nn
import numpy as np
import PIL
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from IPython.display import display, clear_output
from itertools import islice
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from torch import autocast
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

from k_diffusion.sampling import sample_lms
from k_diffusion.external import CompVisDenoiser

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model = model.half().to(device)
    model.eval()
    return model


class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale

class config():
    def __init__(self):
        self.ckpt = checkpoint_model_file
        self.config = 'configs/stable-diffusion/v1-inference.yaml'
        self.ddim_eta = 0.0
        self.ddim_steps = 100
        self.fixed_code = True
        self.init_img = None
        self.n_iter = 1
        self.n_samples = 1
        self.outdir = ""
        self.precision = 'autocast'
        self.prompt = ""
        self.sampler = 'klms'
        self.scale = 7.5
        self.seed = 42
        self.strength = 0.75 # strength for noising/unnoising. 1.0 corresponds to full destruction of information in init image
        self.H = 512
        self.W = 512
        self.C = 4
        self.f = 8
      
def load_img(path, shape):
    if path.startswith('http://') or path.startswith('https://'):
        image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
    else:
        if os.path.isdir(path):
            files = [file for file in os.listdir(path) if file.endswith('.png') or file .endswith('.jpg')]
            path = os.path.join(path, random.choice(files))
            print(f"Chose random init image {path}")
        image = Image.open(path).convert('RGB')
    image = image.resize(shape, resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float16) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

opt = config()
config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")
model = model.to(device)
batch_idx = 0
sample_idx = 0

def generate(opt):
    gc.collect()
    torch.cuda.empty_cache()
    
    global sample_idx
    seed_everything(opt.seed)
    os.makedirs(opt.outdir, exist_ok=True)

    if opt.sampler == 'plms':
        sampler = PLMSSampler(model)
    else:
        sampler = DDIMSampler(model)

    model_wrap = CompVisDenoiser(model)       
    batch_size = opt.n_samples
    prompt = opt.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]
    init_latent = None

    if opt.init_img != None and opt.init_img != '':
        init_image = load_img(opt.init_img, shape=(opt.W, opt.H)).to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

    sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

    t_enc = int(opt.strength * opt.ddim_steps)

    start_code = None
    if opt.fixed_code and init_latent == None:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)

    images = []
    precision_scope = autocast if opt.precision == "autocast" else nullcontext
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for n in range(opt.n_iter):
                    for prompts in data:
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)

                        if init_latent != None:
                            z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                            samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                    unconditional_conditioning=uc,)
                        else:

                            if opt.sampler == 'klms':
                                print("Using KLMS sampling")
                                shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                sigmas = model_wrap.get_sigmas(opt.ddim_steps)
                                model_wrap_cfg = CFGDenoiser(model_wrap)
                                x = torch.randn([opt.n_samples, *shape], device=device) * sigmas[0]
                                extra_args = {'cond': c, 'uncond': uc, 'cond_scale': opt.scale}
                                samples = sample_lms(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=False)
                            else:
                                shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                samples, _ = sampler.sample(S=opt.ddim_steps,
                                                                conditioning=c,
                                                                batch_size=opt.n_samples,
                                                                shape=shape,
                                                                verbose=False,
                                                                unconditional_guidance_scale=opt.scale,
                                                                unconditional_conditioning=uc,
                                                                eta=opt.ddim_eta,
                                                                x_T=start_code)

                        x_samples = model.decode_first_stage(samples)
                        x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                        for x_sample in x_samples:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            images.append(Image.fromarray(x_sample.astype(np.uint8)))
                            filepath = os.path.join(opt.outdir, f"{batch_name}({batch_idx})_{sample_idx:04}.png")
                            print(f"Saving to {filepath}")
                            Image.fromarray(x_sample.astype(np.uint8)).save(filepath)
                            sample_idx += 1
    return images


Loading model from /content/gdrive/MyDrive/stable-diffusion/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.19.mlp.fc2.weight', 'vision_model.encoder.layers.18.self_attn.out_proj.bias', 'text_projection.weight', 'vision_model.encoder.layers.8.self_attn.q_proj.bias', 'vision_model.encoder.layers.13.layer_norm1.bias', 'vision_model.encoder.layers.2.self_attn.out_proj.weight', 'vision_model.encoder.layers.4.mlp.fc2.weight', 'vision_model.encoder.layers.21.self_attn.k_proj.weight', 'vision_model.encoder.layers.11.self_attn.v_proj.bias', 'vision_model.encoder.layers.0.layer_norm2.bias', 'vision_model.encoder.layers.19.self_attn.v_proj.bias', 'vision_model.encoder.layers.18.layer_norm2.bias', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.encoder.layers.7.self_attn.v_proj.weight', 'vision_model.encoder.layers.12.self_attn.q_proj.bias', 'vision_model.encoder.layers.22.self_attn.q_proj.bias', 'vision_model.encoder.layers.16.layer_norm1.bi

In [ ]:
# Prompt Randomizer code by @ai_curio
file = open("/content/google-10000-english/google-10000-english-usa-no-swears-medium.txt", "r")
med_list = file.read().splitlines()
file.close()
file = open("/content/google-10000-english/google-10000-english-usa-no-swears-long.txt", "r")
long_list = file.read().splitlines()
file.close()
heavy_list = med_list + long_list
heavy_list.remove("beastality")

term_list = ["8k resolution", "pencil sketch", "8k 3d", "creative commons attribution", "deviantart", "cryengine", "unreal engine", "conceptart", "photoillustration", "pixiv"]
term_list = term_list + ["flickr", "artstation hd", "behance hd", "hdr", "anime", "filmic", "stock photo", "ambient occlusion", "global illumination", "chalk art", "low poly"]
term_list = term_list + ["polycount", "acryllic art", "hyperrealism", "zbrush central", "rendered in cinema4d", "rendered in maya", "photo taken with nikon d750", "tilt shift", "mixed media"]
term_list = term_list + ["depth of field", "dslr", "detailed painting", "volumetric lighting", "storybook illustration", "unsplash contest winner", "ultrafine detail", "vfxfriday"]
term_list = term_list + ["20 megapixels", "photo taken with fugifilm superia", "photo taken with ektachrome", "matte painting", "painting", "moody painting"]
term_list = term_list + ["matte painting", "painting", "moody painting", "oil painting", "watercolor painting", "pastel painting", "acrylic painting"]*4
term_list = term_list + ["watercolor", "cg society", "child's drawing", "marble sculpture", "airbrush art", "renaissance painting", "velvia", "provia", "photo taken with provia"]
term_list = term_list + ["prerendered graphics", "critereon collection", "dye-transfer", "stipple", "parallax", "bryce 3d", "terragen", "bokeh", "1990s, 1995"]
term_list = term_list + ["1970s, 1975", "1920s, 1925", "charcoal drawing", "commission for", "furaffinity", "flat shading", "ink drawing", "artwork", "oil on canvas"]
term_list = term_list + ["macro photography", "hall of mirrors", "polished", "sunrays shine upon it", "aftereffects", "iridescent", "film", "datamosh", "holographic", "dutch golden age"]
term_list = term_list + ["digitally enhanced", "national geographic photo", "associated press photo", "matte background", "art on instagram", "digital illustration", "stock photo"]
term_list = term_list + ["aftereffects", "speedpainting", "colorized", "detailed", "psychedelic", "wavy", "groovy", "movie poster", "pop art", "made of cardboard", "made of plastic"]
term_list = term_list + ["woodcut", "american propaganda", "soviet propaganda", "ps1 graphics", "fine art", "hd mod", "photorealistic", "poster art", "constructivism", "impressionism"]
term_list = term_list + ["rtx on", "chiaroscuro", "chiaroscuro", "chiaroscuro", "egyptian art", "masterpiece", "fauvism", "dadaist", "shot on 70mm", "art deco", "academic art", "3840x2160"]
term_list = term_list + ["photocollage", "cubism", "surrealist", "THX sound", "zbrush", "panorama", "smooth", "dc comics", "marvel comics", "ukiyo-e", "flemish baroque", "vray tracing"]
term_list = term_list + ["pixel perfect", "quantum wavetracing", "zbrush central contest winner", "iso 200", "32k HUHD", "photocopy", "tri-x 400 tx", "trending on artstation"]
term_list = term_list + ["featured on artstation", "artstation hq", "trending artstation", "ultra hd", "high quality photo", "instax", "infrared", "lomo", "matte drawing", "matte photo"]
term_list = term_list + ["glowing neon", "xbox 360 graphics", "flickering light", "playstation 5 screenshot", "kodak gold 200", "rough", "maximalist", "minimalist", "kodak ektar"]
term_list = term_list + ["kodak portra", "geometric", "cluttered", "rococo", "ssao", "destructive", "2d game art", "2d platformer game screenshot", "3d game art", "3d platformer game screenshot"]
term_list = term_list + ["windows xp", "y2k aesthetic", "screenshotsaturday"]
term_list = term_list + ["seapunk", "vaporwave", "vaporwave", "vaporwave", "creepypasta", "brutalism", "brutalist", "brutalism", "architecture", "a ritual", "ritual", "hollow-eyed horror", "end of the world", "brutalist", "brutalism", "silent hill"]
term_list = term_list + ["trans rights", "bisexual lighting"]
term_list = term_list + ["red", "orange", "yellow", "green", "blue", "indigo", "violet", "pink", "monochrome"]

def new_prompt():
  random.shuffle(heavy_list)
  random.shuffle(term_list)
  term = term_list[0] + " " + heavy_list[0]
  term = " " + term #+ ":" + str(random.randint(1,100)*0.01)
  return term

In [ ]:

#@title Image creation

#@markdown `batch_name`: name for subfolder and filenames<br>
#@markdown `width_height`: image dimensions<br>
#@markdown `guidance_scale`: strength of text prompt<br>
#@markdown `steps`: number of diffusion steps<br>
#@markdown `num_batch_images`: how many images you want to generate in this batch<br>
#@markdown `sampler`: KLMS is recommended<br>
#@markdown `ddim_eta`: scale of variance from 0.0 to 1.0<br>
#@markdown `seed`: use -1 for random seed or specify number manually<br>
#@markdown `init_image_or_folder`: url or path to an image, or path to a folder to pick random images from<br>
#@markdown `init_strength`: from 0.0 to 1.0 how much the init image is used<br>

#@markdown 

#@markdown Batch settings
batch_name = "20221008_pathfinder" #@param {type:"string"}
width_height = [512, 768] #@param{type: 'raw'}
guidance_scale = 12.0 #@param {type:"number"}
steps = 48 #@param {type:"integer"}
samples_per_batch = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
num_batch_images = 100 #@param {type:"integer"}
sampler = 'klms' #@param ["klms","plms", "ddim"]
ddim_eta = 0.75 #@param {type:"number"}
seed = -1 #@param {type:"integer"}

#@markdown 

#@markdown Init image


init_image_or_folder = "" #@param {type:"string"}
init_strength = -1 #@param {type:"number"}

#@markdown 

#@markdown <b>Prompt Explorer</b><br>
#@markdown SEGMENT_COUNT legend: 1 to 5, select that many segments. 0 is select none (disable this prompt). -1 will use the entire input as is.
ALWAYS_INCLUDE = "close-up of a beautiful angelic female paladin, long wavy dark hair, pale skin, highly symmetrical facial features" #@param {type:"string"}
#@markdown or
use_prompt_randomizer = False #@param {type:"boolean"}
SEGMENTS_1 = "color fantasy portrait, color fantasy horror portrait, color fantasy digital illustration" #@param {type:"string"}
SEGMENT_COUNT_1 = 1 #@param {type:"slider", min:-1, max:5, step:1} 
segment_list_1 = SEGMENTS_1.split(", ")
SEGMENTS_2 = "by beeple, by artgerm, by Zdzislaw Beksinski, by HR Giger, by Moebius, by Greg Rutkowski, by magali villeneuve, by stephan martiniere " #@param {type:"string"}
SEGMENT_COUNT_2 = 3 #@param {type:"slider", min:-1, max:5, step:1} 
segment_list_2 = SEGMENTS_2.split(", ")
SEGMENTS_3 = "trending on cg society, trending on artstation, trending on deviantart, reddit" #@param {type:"string"}
SEGMENT_COUNT_3 = 1 #@param {type:"slider", min:-1, max:5, step:1} 
segment_list_3 = SEGMENTS_3.split(", ")
SEGMENTS_4 = "mannerism, magic realist painting, romantic art, symbolism, surrealism, lowbrow, le fantastique, visionary art, grotesque art, mannerist art, fantasy literature" #@param {type:"string"}
SEGMENT_COUNT_4 = 3 #@param {type:"slider", min:-1, max:5, step:1} 
segment_list_4 = SEGMENTS_4.split(", ")
SEGMENTS_5 = "freakshow, nightmare, ominous, occult, creepy, dark, mysterious, creepypasta, digital painting, HDR, 8k, cgsociety, octane engine, elegant, handsome" #@param {type:"string"}
SEGMENT_COUNT_5 = 3 #@param {type:"slider", min:-1, max:5, step:1} 
segment_list_5 = SEGMENTS_5.split(", ")
prompt = ALWAYS_INCLUDE

  
opt.init_img = init_image_or_folder
opt.ddim_steps = steps
opt.n_iter = 1
opt.n_samples = samples_per_batch
opt.outdir = os.path.join(outputs_path, batch_name)
opt.prompt = prompt
opt.sampler = sampler
opt.scale = guidance_scale
opt.seed = random.randint(0, 2**32) if seed == -1 else seed
opt.strength = max(0.0, min(1.0, 1.0 - init_strength))
opt.W, opt.H = map(lambda x: x - x % 64, (width_height[0], width_height[1])) # resize to integer multiple of 64

if opt.strength >= 1 or init_image_or_folder == None:
    opt.init_img = ""

if opt.init_img != None and opt.init_img != '':
    opt.sampler = 'ddim'

if opt.sampler != 'ddim':
    opt.ddim_eta = 0.0

# save settings
settings = {
    'ddim_eta': ddim_eta,
    'guidance_scale': guidance_scale,
    'init_image': init_image_or_folder,
    'init_strength': init_strength,
    'num_batch_images': num_batch_images,
    'prompt': prompt,
    'sampler': sampler,
    'samples_per_batch': samples_per_batch,
    'seed': opt.seed,
    'steps': steps,
    'width': opt.W,
    'height': opt.H,
    'prompt': prompt
}
os.makedirs(opt.outdir, exist_ok=True)
while os.path.isfile(f"{opt.outdir}/{batch_name}({batch_idx})_settings.txt"):
    batch_idx += 1
with open(f"{opt.outdir}/{batch_name}({batch_idx})_settings.txt", "w+", encoding="utf-8") as f:
    json.dump(settings, f, ensure_ascii=False, indent=4)
sample_idx = 0

for i in range(num_batch_images):
    gc.collect()
    torch.cuda.empty_cache()

    # build the prompt
    random.seed()

    if use_prompt_randomizer:
        segment_pr = new_prompt()
    if SEGMENT_COUNT_1 > 0:
        segment_selection_1 = random.sample(segment_list_1, SEGMENT_COUNT_1)
    elif SEGMENT_COUNT_1 == 0:
        segment_selection_1 = []
    else:
        segment_selection_1 = segment_list_1
    if SEGMENT_COUNT_2 > 0:
        segment_selection_2 = random.sample(segment_list_2, SEGMENT_COUNT_2)
    elif SEGMENT_COUNT_2 == 0:
        segment_selection_2 = []
    else:
        segment_selection_2 = segment_list_2
    if SEGMENT_COUNT_3 > 0:
        segment_selection_3 = random.sample(segment_list_3, SEGMENT_COUNT_3)
    elif SEGMENT_COUNT_3 == 0:
        segment_selection_3 = []
    else:
        segment_selection_3 = segment_list_3
    if SEGMENT_COUNT_4 > 0:
        segment_selection_4 = random.sample(segment_list_4, SEGMENT_COUNT_4)
    elif SEGMENT_COUNT_4 == 0:
        segment_selection_4 = []
    else:
        segment_selection_4 = segment_list_4
    if SEGMENT_COUNT_5 > 0:
        segment_selection_5 = random.sample(segment_list_5, SEGMENT_COUNT_5)
    elif SEGMENT_COUNT_5 == 0:
        segment_selection_5 = []
    else:
        segment_selection_5 = segment_list_5

    PROMPT_LIST = []
    if len(segment_selection_1) > 0: PROMPT_LIST.append(', '.join(segment_selection_1))
    if len(segment_selection_2) > 0: PROMPT_LIST.append(', '.join(segment_selection_2))
    if len(segment_selection_3) > 0: PROMPT_LIST.append(', '.join(segment_selection_3))
    if len(segment_selection_4) > 0: PROMPT_LIST.append(', '.join(segment_selection_4))
    if len(segment_selection_5) > 0: PROMPT_LIST.append(', '.join(segment_selection_5))
    if use_prompt_randomizer:
      PROMPT = segment_pr + ", " + ", ".join(PROMPT_LIST)
    else: 
      PROMPT = ALWAYS_INCLUDE + ", " + ", ".join(PROMPT_LIST)
    settings["prompt"] = PROMPT
    opt.prompt = PROMPT
    
    images = generate(opt)

    #clear_output(wait=True)
    print(f"Used seed: {opt.seed}")
    print(f"Saved to: {opt.outdir}")
    for image in images:
        display(image)
        print(settings["prompt"])
    opt.seed = random.randint(0, 2**32) if seed == -1 else seed
